# Predicting customer churn

Churn prediction--predicting whether a customer will stay or leave a company--is one of the more popular applications of machine learning for business, especially among consulting companies trying to sell their services.

Typically the performance of a churn classifier (0 for customer stays, 1 for customer leaves, i.e. churns) is evaluated by a standard metric such as accuracy, precision, recall or ROC-AUC. In real-life, these metrics can be misleading, as they do not reflect the costs and benefits of the different outcomes being summarized by a given metric.

In the case of churn, these costs and benefits can be made very explicit in terms of the classifier's confusion matrix, https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html.

\begin{equation*}
C =  
\begin{pmatrix}
\mathrm{true \, positives} &  \mathrm{false \, positives}  \\
\mathrm{false \, negatives} &  \mathrm{true \, negatives}
\end{pmatrix},
\end{equation*}

or, more generally, for a classifer with $n$ outcomes, the entries of the confusion matrix $C = (C_{ij})$ are the counts of observations known to be in class $i$ and predicted to be in class $j$.

To calculate a business-relevant metric, we need to know the cost for trying to retain a customer and the benefit of retaining a customer.

## Churn reward: simplest case

The first case we consider is for a single action of sending customers an email. The reward is the revenue from the customer over the next year minux expenses per customer. Let's make the assumptions more explicit, and flag the ones that are reasonable or not as an approximation of reality.

* action $a$ is defined by $a \in (0,1) \leftrightarrow (\mathrm{no\,email\,sent}, \mathrm{email\,sent})$ has a fixed cost for all customers (reasonable),
* there are no costs except the marketing action above (unreasonable)
* revenue $\mathrm{rev}$ is the same for all customers (unreasonable):

\begin{equation*}
\mathrm{rev} = \begin{cases}
0,  & \text{if customer churns} \\
\mathrm{rev}_1, & \text{if customer stays}
\end{cases}
\end{equation*}

The first task is to cast this setup as a *Markov Decision Process*

In [ ]:
import os
from pathlib import Path
import pickle

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelBinarizer, StandardScaler, OneHotEncoder
from sklearn.preprocessing import FunctionTransformer


from sklearn_pandas import DataFrameMapper

import fake_data_for_learning

from risk_learning.config import filenames

%matplotlib inline

## Churn data

In [ ]:
df = pd.read_csv(filenames.fake_churn_rl)
df.head()

## Preprocess

In [ ]:
def id_map(x):
    return x

id_trans = FunctionTransformer(id_map, validate=True)

mapper = DataFrameMapper([
    ('gender', LabelBinarizer()),
    (['age'], StandardScaler()),
    (['profession'], OneHotEncoder()),
    (['action'], id_trans)
])

data = pd.DataFrame(
    mapper.fit_transform(df),
    columns=mapper.transformed_names_
)
data.head()

In [ ]:
with open(filenames.churn_bn, 'rb') as f:
    churn_bn = pickle.load(f)